In [3]:
!pip install bertopic datasets openai datamapplot datasets

In [1]:
from datasets import load_dataset

dataset = load_dataset("maartengr/arxiv_nlp")["train"]

abstracts = dataset["Abstracts"]
titles = dataset["Titles"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("thenlper/gte-small")

embeddings = embedding_model.encode(abstracts,show_progeress  =True)

In [ ]:

embeddings.shape

In [ ]:
from umap import UMAP

umap_model = UMAP(
    n_components = 5 ,
    metric ="cosine",
    min_dist = 0.0,
    random_state =42
)

reduced_embeddings = umap_model.fit_transform(embeddings)

In [ ]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(
    min_cluster_size=50, metric='euclidean', cluster_selection_method='eom'
).fit(reduced_embeddings)

clusters = hdbscan.labels_

print(len(set(clusters)))

In [ ]:
import numpy as np
cluster = 0
for index in np.where(clusters==cluster)[0][:3]:
  print(abstracts[index][:300] + ".../")



In [ ]:
import pandas as pd

reduced_embeddings = UMAP(
    n_components = 2 , min_dist =0.0 , metric='cosine' ,random_state = 42
).fit_transform(embeddings)


df = pd.DataFrame(reduced_embeddings,columns = ["X","Y"])

df["title"]  = titles
df["cluster"] = [str(c) for c in clusters]

clusters_df = df.loc[df.cluster!=-1,:]
outliers_df = df.loc[df.cluster==-1,:]







In [ ]:
import maptplotlib.pyplot as plt
plt.scatter(outliers_df,x ,outliers_df.y , alpha = 0.5 ,s =2 ,c ="green")

plt.scatter(clusters_df.x ,clusters_df.y,c= clusters_df.cluster.astype(int),alpha = 0.6 ,s=2 , cmap ='tab20b')
plt.axis("off")

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(
    embedding_model = embedding_model,
    umap_model = umap_model,
    hdbscan_model = hdbscan_model,
    verbose = True
).fit(abstracts,embeddings)


In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)

In [ ]:
topic_model.find_topics("Topic Modelling")



In [ ]:
topic_model.get_topic(22)

In [ ]:
fig = topic_model.visualize_documents(
    titles,
    reduced_embeddings = reduced_embeddings,
    width =1200,
    hide_annotations =True
)
fig.update_layout(font=dict(size=16))

In [ ]:
topic_model.visualize_barchart()

topic_model.visualize_heatmap(n_clusters= 30)
topic_model.visualize_hierarchy()